# Introduction
This file converts the new cleaned raw dataset into a single merged file that the TFTModel can work on. The work here is done following [`prepareData_Main.py`](../TF2/TFTTF2_ModelDev/prepareData_main.py) and [`data_preparation.py`](../TF2/TFTTF2_ModelDev/data_preparation.py) scripts. However, those two scripts are for the `old dataset`. This notebook is for the `new dataset`. It also removes several assumptions made by the old scripts. I found it too inconvenient to modify the old script to support both.

If you need to change the input feature set, only add that info in the `"data"` section of the `config.json`. This notebook will update the rest (at least feature column mappings and locations) . If you have pivoted dynamic feature and need to melt that date columns, make sure to keep the feature name as string in `"dynamic_features_map"`. If it is already melted and your dynamic file has a `Date` column list or string format both is fine.

In the final output null values are replaced with 0. If you don't want that, comment that out.

### Differences from the old script

|Old script|This notebook|
|---|---|
|Can only keep one feature per static feature file.|Can keep as many features needed per static feature file.|
|One feature per dynamic feature file. | Can handle one or multiple dynamic features per file. |
|Converts static features into dynamic (adds date) then agains drops the dates later|No need to add dates in static features or convert it to dynamic.|
| Left joins features based on `Date` and `FIPS`. However, this may create different merged files depending on the order of input feature files. So even with same feature files we might get very different merged files.| Outer joins features based on `Date` and inner join on `FIPS`. This fixes being dependent on processing sequence like `left` join.| 
| Uses `Population.csv` file as a base for `FIPS`. | Same |
| Uses a random first csv file as a base for county `Name`s. Uses `Name` as id in config. | Uses `Population.csv` file as a base for `County` names. Uses `County` as id. Since `Name` would have ambiguous meaning.|
| Re-implements custom MinMaxScaler. | Uses a MinMaxScaler from sklearn library. |
| Scales down only the cluster choosen by Rurality. Local scaling. More like `RMSE` of their deviation ratio from the minimum value for that particular cluster. | Same. But probably will change to global scaling later. As that is generally used in practice and will make scores across different clusters dirrectly comparable. Also I am in support of not scaling the target feature to find the actual `RMSE` of covid cases.|
| Old data uses start date 2020-3-1 and end date 2021-11-22. Can't update further due to lack of valid data source. | New dataset uses start date 2020-3-1 and end date 2021-11-22. Can be updated further till 2022-2-5 for now. |

# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os, json
import math

# Setup storage

## Input
If running on colab add root to both dataPath and configPath. Note that this config.json is different from the config.json in TF2 folder as that is for the old dataset.

In [2]:
# folder where the cleaned feature file are at
dataPath = "cleaned"

configPath = 'config.json'
with open(configPath) as inputFile:
    config = json.load(inputFile)
    inputFile.close()

config = config['TFTparams']['data']

## Output
Creates two files
* outputPathTotal
  * path where the merged csv files will be dumped
  * contains all counties
  * can be reused to create further clusters

* outputPathFinal
  * only contains counties selected by current Rurality cut in config.json

In [3]:
outputPathTotal = 'TFTdfTotal.csv'
outputPathFinal = 'TFTdfCurrent.csv'

# Static features
## Static features mapping

In [4]:
# map between csv filename and feature columns extracted from that file
# each file must have FIPS column, no index
static_features_map = config['static_features_map']
static_features = []
for value in static_features_map.values():
    static_features.extend(value)

print(f'Static features {static_features}')

id_columns = ['FIPS', 'County']

Static features ['Total Population', 'Population Density', 'Population 55+', '% Fair or Poor Health', '% Adults with Obesity', '% Flu Vaccinated']


## Read base static feature
All other static features will be merged on this. County names are also extracted from this base feature file.

In [5]:
# We'll use population file as the base and take the county names from it
# then merge other files to it
def read_feature_file(file_name):
    return pd.read_csv(os.path.join(dataPath, f'{file_name}'))

support_file = config['support']['Population']
static_df = read_feature_file(support_file)
static_df = static_df[id_columns + static_features_map[support_file]]

locs = static_df['FIPS'].nunique()
print(f'Unique counties present {locs}')

Unique counties present 3140


## Merge

In [6]:
for file_name in static_features_map.keys():
    if file_name == support_file: continue

    feature_df = read_feature_file(file_name)
    print(f'Merging feature {file_name} with length {feature_df.shape[0]}')
    static_df = static_df.merge(feature_df[['FIPS'] + static_features_map[file_name]], how='inner', on='FIPS')

print(f"\nMerged static features have {static_df['FIPS'].nunique()} counties")
static_df.head()

Merging feature Health rank measure.csv with length 3142

Merged static features have 3140 counties


,FIPS,County,Total Population,Population Density,Population 55+,% Fair or Poor Health,% Adults with Obesity,% Flu Vaccinated
0,1001,Autauga County,55869.0,36.287947,16190.0,19.839179,33.0,42.0
1,1003,Baldwin County,223234.0,54.215293,79245.0,16.460675,30.0,46.0
2,1005,Barbour County,24686.0,10.769826,8110.0,29.841500,41.2,39.0
3,1007,Bibb County,22394.0,13.890616,6639.0,23.853284,37.4,40.0
4,1009,Blount County,57826.0,34.624193,18460.0,21.985610,33.0,40.0


# Dynamic features
## Dynamic feature mapping

In [7]:
def valid_date(date):
    try:
        pd.to_datetime(date)
        return True
    except:
        return False

# notice: no need to add .csv to filename
# {feature_file_name: feature_name}
dynamic_features_map = config['dynamic_features_map']

dynamic_features = []
for value in dynamic_features_map.values():
    if type(value)==str:
        dynamic_features.append(value)
    else:
        dynamic_features.extend(value)
print(dynamic_features)

['workplaces_percent_change_from_baseline', 'Administered_Dose1_Recip', 'Series_Complete_Yes', 'Testing']


In [8]:
first_date = pd.to_datetime(config['support']['FirstDate'])

# determined following mobility bts
# that feature has the min end date available across all feature files
# which for now is 2022-02-05. 
last_date = pd.to_datetime(config['support']['LastDate'])

In [9]:
dynamic_df = None
merge_keys = ['FIPS', 'Date']

for file_name in dynamic_features_map.keys():
    print(f'Reading {file_name}')
    df = read_feature_file(file_name)

    # only needed if you have empty column names in the feature file
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # check whether the Date column has been pivoted
    if 'Date' not in df.columns:
         # technically this should be set of common columns
        id_vars = [col for col in df.columns if not valid_date(col)]
        df = df.melt(
            id_vars= id_vars,
            var_name='Date', value_name=dynamic_features_map[file_name]
        ).reset_index(drop=True)

    # can be needed as some feature files may have different date format
    df['Date'] = pd.to_datetime(df['Date'])
    print(f'Min date {df["Date"].min()}, max date {df["Date"].max()}')
    df = df[(df['Date'] >= first_date) & (df['Date'] <= last_date)]

    print(f'Length {df.shape[0]}.')

    if dynamic_df is None: dynamic_df = df
    else:
        # if a single file has multiple features
        if type(dynamic_features_map[file_name])==list:
            selected_columns = merge_keys + dynamic_features_map[file_name]
        else:
            selected_columns = merge_keys + [dynamic_features_map[file_name]]

        # using outer to keep the union of dates 
        # as vaccination dates are not available before late in 2020
        dynamic_df = dynamic_df.merge(df[selected_columns], how='outer',on=merge_keys)

        # however, we don't need to keep mismatch of FIPS
        dynamic_df = dynamic_df[~dynamic_df['FIPS'].isna()]

print(f'Total dynamic feature shape {dynamic_df.shape}')
dynamic_df.head()


Reading Mobility google.csv
Min date 2020-02-15 00:00:00, max date 2022-03-30 00:00:00
Length 1587062.
Reading Vaccination.csv
Min date 2020-12-13 00:00:00, max date 2022-04-03 00:00:00
Length 1112280.
Reading Testing.csv
Min date 2020-01-22 00:00:00, max date 2022-03-27 00:00:00
Length 1946560.
Total dynamic feature shape (2010742, 6)


,FIPS,Date,workplaces_percent_change_from_baseline,Administered_Dose1_Recip,Series_Complete_Yes,Testing
0,1001,2020-03-01,3.0,NaN,NaN,0.0
1,1001,2020-03-02,5.0,NaN,NaN,0.0
2,1001,2020-03-03,4.0,NaN,NaN,0.0
3,1001,2020-03-04,3.0,NaN,NaN,0.0
4,1001,2020-03-05,3.0,NaN,NaN,0.0


# Target feature
Converts cumulative covid cases into daily cases. Also remove outliers. For now only handling one target here.

In [10]:
# cases
target_column = list(config['targets'].keys())[0]

# read cumulative cases.csv
target_df = read_feature_file(config['targets'][target_column])
target_df['Date'] = pd.to_datetime(target_df['Date'])

# Merge all together

In [11]:
total_df = target_df.merge(dynamic_df, how='inner', on=['FIPS', 'Date'])
total_df = static_df.merge(total_df, how='inner', on='FIPS')
total_df = total_df.reset_index(drop=True)

print(total_df.shape)
total_df.head()

(1981188, 14)


,FIPS,County,Total Population,Population Density,Population 55+,% Fair or Poor Health,% Adults with Obesity,% Flu Vaccinated,Date,Cases,workplaces_percent_change_from_baseline,Administered_Dose1_Recip,Series_Complete_Yes,Testing
0,1001,Autauga County,55869.0,36.287947,16190.0,19.839179,33.0,42.0,2020-03-01,0.0,3.0,NaN,NaN,0.0
1,1001,Autauga County,55869.0,36.287947,16190.0,19.839179,33.0,42.0,2020-03-02,0.0,5.0,NaN,NaN,0.0
2,1001,Autauga County,55869.0,36.287947,16190.0,19.839179,33.0,42.0,2020-03-03,0.0,4.0,NaN,NaN,0.0
3,1001,Autauga County,55869.0,36.287947,16190.0,19.839179,33.0,42.0,2020-03-04,0.0,3.0,NaN,NaN,0.0
4,1001,Autauga County,55869.0,36.287947,16190.0,19.839179,33.0,42.0,2020-03-05,0.0,3.0,NaN,NaN,0.0


In [12]:
def missing_percentage(df):
    return df.isnull().mean().round(4).mul(100).sort_values(ascending=False)

missing_percentage(total_df)

Administered_Dose1_Recip                   47.21
Series_Complete_Yes                        45.46
workplaces_percent_change_from_baseline    21.45
Testing                                     1.81
% Flu Vaccinated                            0.54
FIPS                                        0.00
County                                      0.00
Total Population                            0.00
Population Density                          0.00
Population 55+                              0.00
% Fair or Poor Health                       0.00
% Adults with Obesity                       0.00
Date                                        0.00
Cases                                       0.00
dtype: float64

In [13]:
total_df = total_df.fillna(0)

# dump the total merged data
# total_df.to_csv(outputPathTotal, index=False)

# Uncomment if only starting from here
# total_df = pd.read_csv(outputPathTotal)

# Cut based on rurality median

In [14]:
MADRANGE = config['support']['MADRange']
RURRANGE = config['support']['RuralityRange']

# fails to read on unicode
rur = pd.read_csv(os.path.join(dataPath, config['support']["Rurality"]), encoding = 'latin1')

locs = rur.FIPS

if -1 in RURRANGE:
    print('No Median Rurality Cut')
    lost = []
else:
    locs = rur[(rur['Median'] >= RURRANGE[0]) & (rur['Median'] <= RURRANGE[1])].FIPS
    lost = rur[~((rur['Median'] >= RURRANGE[0]) & (rur['Median'] <= RURRANGE[1]))].FIPS
    rur = rur[rur['FIPS'].isin(locs)]

print('Lost number of locations from median cut ' + str(len(lost)))
print('Remaining number of locations from median cut ' + str(len(locs)))

if -1 in MADRANGE:
    print('No MAD cut')
    lost = []
else:
    locs = rur[(rur['MAD'] >= MADRANGE[0]) & (rur['MAD'] < MADRANGE[1])].FIPS
    lost = rur[~((rur['MAD'] >= MADRANGE[0]) & (rur['MAD'] < MADRANGE[1]))].FIPS

print('Lost Num Locations from MAD Cut ' + str(len(lost)))
print('Remaining Num Locations from MAD Cut ' + str(len(locs)))

print('#' * 50)
print('Final Location Count: ' + str(len(locs)))

Lost number of locations from median cut 3039
Remaining number of locations from median cut 182
Lost Num Locations from MAD Cut 103
Remaining Num Locations from MAD Cut 79
##################################################
Final Location Count: 79


In [15]:
# only keep the selected counties
total_df = total_df[total_df['FIPS'].isin(locs)].reset_index(drop=True)
total_df.shape

(49897, 14)

# Scaling

In [16]:
all_features = static_features + dynamic_features
feature_scaler = MinMaxScaler()
total_df[all_features] = feature_scaler.fit_transform(total_df[all_features])

target_scaler = MinMaxScaler()
total_df[[target_column]] = target_scaler.fit_transform(np.sqrt(total_df[[target_column]]))

# check for any inconsistency
total_df.head()

,FIPS,County,Total Population,Population Density,Population 55+,% Fair or Poor Health,% Adults with Obesity,% Flu Vaccinated,Date,Cases,workplaces_percent_change_from_baseline,Administered_Dose1_Recip,Series_Complete_Yes,Testing
0,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-01,0.0,0.664336,0.0,0.0,0.0
1,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-02,0.0,0.650350,0.0,0.0,0.0
2,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-03,0.0,0.657343,0.0,0.0,0.0
3,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-04,0.0,0.622378,0.0,0.0,0.0
4,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-05,0.0,0.643357,0.0,0.0,0.0


# Add embedding

In [17]:
def add_embeddings(data):
    def LinearLocationEncoding(TotalLoc):
        linear = np.empty(TotalLoc, dtype=float)
        for i in range(0, TotalLoc):
            linear[i] = float(i) / float(TotalLoc)
        return linear

    def LinearTimeEncoding(Dateslisted):
        Firstdate = Dateslisted[0]
        numtofind = len(Dateslisted)
        dayrange = (Dateslisted[numtofind - 1] - Firstdate).days + 1
        linear = np.empty(numtofind, dtype=float)
        for i in range(0, numtofind):
            linear[i] = float((Dateslisted[i] - Firstdate).days) / float(dayrange)
        return linear

    def P2TimeEncoding(numtofind):
        P2 = np.empty(numtofind, dtype=float)
        for i in range(0, numtofind):
            x = -1 + 2.0 * i / (numtofind - 1)
            P2[i] = 0.5 * (3 * x * x - 1)
        return P2

    def P3TimeEncoding(numtofind):
        P3 = np.empty(numtofind, dtype=float)
        for i in range(0, numtofind):
            x = -1 + 2.0 * i / (numtofind - 1)
            P3[i] = 0.5 * (5 * x * x - 3) * x
        return P3

    def P4TimeEncoding(numtofind):
        P4 = np.empty(numtofind, dtype=float)
        for i in range(0, numtofind):
            x = -1 + 2.0 * i / (numtofind - 1)
            P4[i] = 0.125 * (35 * x * x * x * x - 30 * x * x + 3)
        return P4

    def WeeklyTimeEncoding(Dateslisted):
        numtofind = len(Dateslisted)
        costheta = np.empty(numtofind, dtype=float)
        sintheta = np.empty(numtofind, dtype=float)
        for i in range(0, numtofind):
            j = Dateslisted[i].date().weekday()
            theta = float(j) * 2.0 * math.pi / 7.0
            costheta[i] = math.cos(theta)
            sintheta[i] = math.sin(theta)
        return costheta, sintheta

    # Set up linear location encoding for all of the data
    LLE = LinearLocationEncoding(config["support"]["Nloc"])

    for idx, i in enumerate(data['FIPS'].unique()):
        data.loc[data['FIPS'] == i, 'LinearSpace'] = LLE[idx]

    # Set up constant encoding
    data['Constant'] = 0.5

    # Set up linear time encoding
    dates = pd.to_datetime(data['Date'].unique())

    LTE = LinearTimeEncoding(dates)
    P2E = P2TimeEncoding(len(dates))
    P3E = P3TimeEncoding(len(dates))
    P4E = P4TimeEncoding(len(dates))

    CosWeeklyTE, SinWeeklyTE = WeeklyTimeEncoding(dates)

    for idx, i in enumerate(dates):
        data.loc[data['Date'] == i, 'LinearTime'] = LTE[idx]
        data.loc[data['Date'] == i, 'P2Time'] = P2E[idx]
        data.loc[data['Date'] == i, 'P3Time'] = P3E[idx]
        data.loc[data['Date'] == i, 'P4Time'] = P4E[idx]
        data.loc[data['Date'] == i, 'CosWeekly'] = CosWeeklyTE[idx]
        data.loc[data['Date'] == i, 'SinWeekly'] = SinWeeklyTE[idx]

    return data

In [18]:
total_df['Date'] = pd.to_datetime(total_df['Date'])
total_df['TimeFromStart'] = (total_df['Date'] - first_date).dt.days

pre_columns = total_df.columns
total_df = add_embeddings(total_df)
known_future_features = [col for col in total_df.columns if col not in pre_columns]

print(total_df.shape)
total_df.head()

(49897, 23)


,FIPS,County,Total Population,Population Density,Population 55+,% Fair or Poor Health,% Adults with Obesity,% Flu Vaccinated,Date,Cases,...,Testing,TimeFromStart,LinearSpace,Constant,LinearTime,P2Time,P3Time,P4Time,CosWeekly,SinWeekly
0,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-01,0.0,...,0.0,0,0.0,0.5,0.000000,1.000000,-1.000000,1.000000,0.623490,-0.781831
1,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-02,0.0,...,0.0,1,0.0,0.5,0.001582,0.990506,-0.981058,0.968530,1.000000,0.000000
2,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-03,0.0,...,0.0,2,0.0,0.5,0.003165,0.981043,-0.962266,0.937508,0.623490,0.781831
3,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-04,0.0,...,0.0,3,0.0,0.5,0.004747,0.971609,-0.943624,0.906932,-0.222521,0.974928
4,1031,Coffee County,0.408191,0.348635,0.300025,0.457302,0.483283,0.487179,2020-03-05,0.0,...,0.0,4,0.0,0.5,0.006329,0.962206,-0.925131,0.876798,-0.900969,0.433884


In [19]:
# ['FIPS', 'County' + 'Date', 'TimeFromStart', target_column]
# +static_features + dynamic_features + known_future_features

total_df.to_csv(outputPathFinal, index=False)
total_df.columns

Index(['FIPS', 'County', 'Total Population', 'Population Density',
       'Population 55+', '% Fair or Poor Health', '% Adults with Obesity',
       '% Flu Vaccinated', 'Date', 'Cases',
       'workplaces_percent_change_from_baseline', 'Administered_Dose1_Recip',
       'Series_Complete_Yes', 'Testing', 'TimeFromStart', 'LinearSpace',
       'Constant', 'LinearTime', 'P2Time', 'P3Time', 'P4Time', 'CosWeekly',
       'SinWeekly'],
      dtype='object')

# Update config.json
Make sure your config.json is consistent with these info. Maybe we can directly update config from this notebook or create a separate config for model in future.

In [20]:
static_locs = [i for i in range(len(static_features))]
print(f'static locs: {static_locs}')

start = len(static_features) + len(dynamic_features)
future_locs = [i for i in range(start, start + len(known_future_features))]
print(f'future locs: {future_locs}')

target_loc = start + len(known_future_features)
print(f'target loc: {target_loc}. total input {target_loc+1}')

print(f'col_mappings: Static {static_features}')
print(f'col_mappings: Future {known_future_features}')
print(f'col_mappings: Known Regular  {static_features + dynamic_features}')

static locs: [0, 1, 2, 3, 4, 5]
future locs: [10, 11, 12, 13, 14, 15, 16, 17]
target loc: 18. total input 19
col_mappings: Static ['Total Population', 'Population Density', 'Population 55+', '% Fair or Poor Health', '% Adults with Obesity', '% Flu Vaccinated']
col_mappings: Future ['LinearSpace', 'Constant', 'LinearTime', 'P2Time', 'P3Time', 'P4Time', 'CosWeekly', 'SinWeekly']
col_mappings: Known Regular  ['Total Population', 'Population Density', 'Population 55+', '% Fair or Poor Health', '% Adults with Obesity', '% Flu Vaccinated', 'workplaces_percent_change_from_baseline', 'Administered_Dose1_Recip', 'Series_Complete_Yes', 'Testing']


In [21]:
# read the config file again
with open(configPath) as inputFile:
    config = json.load(inputFile)
    inputFile.close()

config["TFTparams"]["static_locs"] = static_locs
config["TFTparams"]["future_locs"] = future_locs
config["TFTparams"]["target_loc"] = [target_loc]
config["TFTparams"]["total_inputs"] = target_loc + 1

config["TFTparams"]["col_mappings"]["Static"] = static_features

# this notebook doesn't support multiple target columns yet
config["TFTparams"]["col_mappings"]["Target"] = [target_column]
config["TFTparams"]["col_mappings"]["Future"] = known_future_features
config["TFTparams"]["col_mappings"]["Known Regular"] = static_features + dynamic_features

# dump the json config
with open(configPath, 'w') as outputFile:
    json.dump(config, outputFile, indent=4)
    outputFile.close()

# Run TFT model

Now go to `TF2/TFTTF2_ModelDev` and run the following to run TFT on this new dataset

```python
python main.py -p "../../dataset_new/config.json" -c checkpoints -d "../../dataset_new/TFTdfCurrent.csv"
```